In [120]:
import json
import joblib
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier



import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import fairlearn

In [121]:
df=pd.read_csv('two-years-recid.csv')#.set_index('id')
df

,id,name,sex,dob,age,race,c_jail_in,c_jail_out,c_case_number,c_offense_date,...,r_case_number,r_offense_date,r_charge_degree,r_charge_desc,is_violent_recid,vr_case_number,vr_offense_date,vr_charge_degree,vr_charge_desc,two_year_recid
0,1,steven lux,Male,1953-06-15,62,Caucasian,2013-01-05 04:35:31,2013-01-07 03:18:03,13000208CF10A,2013-01-05,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
1,2,andre small,Male,1987-10-01,28,African-American,2013-04-02 12:18:46,2013-04-04 07:54:22,13006354MM10A,2013-04-02,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
2,3,willie gray,Male,1959-01-12,57,African-American,2014-10-31 12:02:01,2014-10-31 01:47:05,14040148MU10A,2014-10-30,...,15043364TC20A,2015-07-23,(M2),Driving License Suspended,0,NaN,NaN,NaN,NaN,1
3,4,nickson marcellus,Male,1996-07-11,19,African-American,2014-01-23 03:19:30,2014-01-23 01:04:34,13017969CF10A,NaN,...,16000241MM20A,2016-01-04,(M1),Possess Cannabis/20 Grams Or Less,0,NaN,NaN,NaN,NaN,1
4,5,patria barnes,Female,1978-06-06,37,Other,2013-12-08 01:55:28,2013-12-09 02:00:59,13022717MM10A,2013-12-07,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,6110,seccunda davis,Male,1987-05-13,28,African-American,2013-08-18 07:25:24,2013-08-19 09:01:42,13015644MM10A,2013-08-18,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6110,6111,mark montgomery,Male,1985-11-03,30,African-American,2013-03-23 01:32:34,2013-03-28 09:37:27,13005696MM10A,2013-03-23,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6111,6112,erica johnson,Female,1982-06-23,33,Caucasian,2013-09-29 09:25:30,2013-09-30 09:59:37,13013661CF10A,2013-09-29,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0
6112,6113,barry williams,Male,1988-04-22,27,African-American,2013-10-30 03:40:14,2013-12-07 01:53:45,13004112MM10A,NaN,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0


In [122]:
## REMOVING ROWS FOR GOAL

In [123]:
df = df.dropna(subset=['c_jail_out'])

# Suppose df_test has some different values, so we'll modify it directly
# Convert columns to datetime if they are not already
df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])

df['r_offense_date'] = pd.to_datetime(df['r_offense_date'])


#df.drop(columns=['two_year_recid'], inplace=True)
### Clean dataset to target variable

# Calculate the difference between 'r_offense_date' and 'c_jail_out'
df['offense_jail_duration'] = (df['r_offense_date'] - df['c_jail_out']).dt.days

# Create a new column based on your conditions
df['two_year_recid'] = ((df['is_recid'] == 1) & (df['offense_jail_duration'] <= 365 * 2)).astype(int)

#
df['is_recid']=df['two_year_recid']

/tmp/ipykernel_2432/3120417259.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
/tmp/ipykernel_2432/3120417259.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])
/tmp/ipykernel_2432/3120417259.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [124]:
# Convert dates to datetime
df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])
df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])

# Swap c_jail_in and c_jail_out where c_jail_out is before c_jail_in and the difference is less than one day
swap_mask = (df['c_jail_out'] < df['c_jail_in']) & ((df['c_jail_in'] - df['c_jail_out']).dt.days < 1)
df.loc[swap_mask, ['c_jail_in', 'c_jail_out']] = df.loc[swap_mask, ['c_jail_out', 'c_jail_in']].values


/tmp/ipykernel_2432/2355313829.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])
/tmp/ipykernel_2432/2355313829.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['c_jail_out'] = pd.to_datetime(df['c_jail_out'])
/tmp/ipykernel_2432/2355313829.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [20]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the preprocess_data function
def preprocess_data(df):
    df = df.astype({
        "id": str,
        "name": str,
        "sex": str,
        "dob": str,
        "race": str,
        "juv_fel_count": int,
        "juv_misd_count": int,
        "juv_other_count": int,
        "priors_count": int,
        "c_case_number": str,
        "c_charge_degree": str,
        "c_charge_desc": str,
        "c_offense_date": str,
        "c_arrest_date": str,
        "c_jail_in": str
    })
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(0)  # Fill NaN with 0
    df['time_in_jail'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days  # Duration in jail till today
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    
    return df

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Apply preprocessing to both training and test sets
df_train = preprocess_data(df_train)
df_test = preprocess_data(df_test)

# Define features
all_features = [
    "sex",
    "race",
    "juv_fel_count",
    "juv_misd_count",
    "juv_other_count",
    "priors_count",
    "c_charge_degree",
    "c_charge_desc",
    "age_birth",
    "age_at_unified_date",
    "time_offense_arrest",
    "time_in_jail",
    "total_juv_crimes"
]

numerical_features = [
    "juv_fel_count",
    "juv_misd_count",
    "juv_other_count",
    "priors_count",
    "age_birth",
    "age_at_unified_date",
    "time_offense_arrest",
    "time_in_jail",
    "total_juv_crimes"
]

categorical_features = [
    "sex",
    "race",
    "c_charge_degree"
]

text_features = ["c_charge_desc"]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('txt', text_transformer, 'c_charge_desc')
    ]
)

# Define the model pipeline with LogisticRegression and specified hyperparameters
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        C=1.0,
        penalty='l2',
        random_state=42,
        max_iter=10000  # Adding max_iter to ensure convergence
    ))
])

# Train the model
pipeline.fit(df_train[all_features], df_train[target])

# Evaluate on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')


Accuracy: 0.71
Precision: 0.70
Recall: 0.61
F1-Score: 0.65
ROC-AUC: 0.77
Confusion Matrix:
[[505 139]
 [204 323]]


## Baseline

In [288]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
import joblib

# Define the categorize_charge function
# Frequency encoding
def frequency_encoding(df, column):
    freq_encoding = df[column].value_counts(normalize=True)
    df[f'{column}_freq'] = df[column].map(freq_encoding)
    return df


# Define the preprocess_data function
def preprocess_data(df):
    df = df.astype({
        "id": str,
        "name": str,
        "sex": str,
        "dob": str,
        "race": str,
        "juv_fel_count": int,
        "juv_misd_count": int,
        "juv_other_count": int,
        "priors_count": int,
        "c_case_number": str,
        "c_charge_degree": str,
        "c_charge_desc": str,
        "c_offense_date": str,
        "c_arrest_date": str,
        "c_jail_in": str
    })
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['c_jail_in'] - df['dob']).dt.days // 365
    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(0)  # Fill NaN with 0
    df['time_since_jail'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days  # Duration in jail till today
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Calculate time_in_jail as the difference between c_jail_in and unified_date
    df['time_to_jail'] = (df['c_jail_in'] - df['unified_date']).dt.days.fillna(0)

    # Categorize charge descriptions
    df['charge_category'] = df['c_charge_desc'].apply(categorize_charge)
    
    ## Create age bins
    #bins = [18, 25, 35, 45, 55, 65, 100]
    #labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    #df['age_group'] = pd.cut(df['age_at_unified_date'], bins=bins, labels=labels, right=False)
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek
    df['arrest_month'] = df['c_arrest_date'].dt.month
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek

    # Interaction features
    #df['fel_misd_interaction'] = df['juv_fel_count'] * df['juv_misd_count']
    
    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Apply preprocessing to both training and test sets
df_train = preprocess_data(df_train)
df_test = preprocess_data(df_test)

# Define features
all_features = [
    "sex", "race",
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
    #"charge_category",
    "c_charge_desc",
    "age_at_unified_date",
    #"age_group",
    #"time_offense_arrest", 
    #"time_since_jail", 
    "time_to_jail",
    "total_juv_crimes",
    "offense_month", "offense_day_of_week", "arrest_month", "arrest_day_of_week",
    #"fel_misd_interaction"
]

numerical_features = [
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count", 
    "total_adult_crimes",
    #"time_offense_arrest", 
    #"time_in_jail", 
    #"time_since_jail",
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    #"fel_misd_interaction",
    "age_at_unified_date"
]

categorical_features = [
    #"sex", 
    #"race", 
    "c_charge_degree", 
    #"charge_category", 
    "c_charge_desc",
    #"age_group"

]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Update the pipeline with the best model and parameters
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample':0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')


Accuracy: 0.69
Precision: 0.68
Recall: 0.58
F1-Score: 0.63
ROC-AUC: 0.74
Confusion Matrix:
[[503 141]
 [222 305]]


## Hyperparameter tuning for ROC

In [155]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# Define a dictionary of models with their respective parameter grids for hyperparameter tuning
models = {
    'Logistic Regression': {
        'model': LogisticRegression(solver='liblinear'),  # Change solver here
        'params': {
            'classifier__C': [0.1, 1.0, 10.0],
            'classifier__penalty': ['l1', 'l2']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4],
            'classifier__bootstrap': [True, False]
        }
    },
    'Support Vector Machine': {
        'model': SVC(),
        'params': {
            'classifier__C': [0.1, 1.0, 10.0],
            'classifier__kernel': ['linear', 'rbf']
        }
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 1.0]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 1.0],
            'classifier__max_depth': [3, 5, 7]
        }
    },
    'K-Nearest Neighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance']
        }
    },
    'Multi-layer Perceptron': {
        'model': MLPClassifier(),
        'params': {
            'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50, 25)],
            'classifier__activation': ['logistic', 'relu'],
            'classifier__alpha': [0.0001, 0.001, 0.01]
        }
    },
    'Extra Trees': {
        'model': ExtraTreesClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4],
            'classifier__bootstrap': [True, False]
        }
    }
}

for name, model_info in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model_info['model'])])
    grid_search = GridSearchCV(pipeline, model_info['params'], cv=5, scoring=['roc_auc'], refit='roc_auc', error_score='raise')
    grid_search.fit(X_train, y_train)
    print(f'Best parameters for {name}: {grid_search.best_params_}')
    print(f'ROC AUC Score: {grid_search.best_score_:.4f}')


Best parameters for Logistic Regression: {'classifier__C': 0.1, 'classifier__penalty': 'l1'}
ROC AUC Score: 0.7672
Best parameters for Decision Tree: {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 10}
ROC AUC Score: 0.7884
Best parameters for Random Forest: {'classifier__bootstrap': True, 'classifier__max_depth': 20, 'classifier__min_samples_leaf': 4, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 300}
ROC AUC Score: 0.8048
Best parameters for Support Vector Machine: {'classifier__C': 1.0, 'classifier__kernel': 'rbf'}
ROC AUC Score: 0.7884
Best parameters for AdaBoost: {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 100}
ROC AUC Score: 0.8062
Best parameters for Gradient Boosting: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 50}
ROC AUC Score: 0.8067
Best parameters for K-Nearest Neighbors: {'classifier__n_neighbors': 7, 'classifier__weights': 'distance'}
ROC 

/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.loca

/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.loca

/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.loca

/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/teresaramoos/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Best parameters for Multi-layer Perceptron: {'classifier__activation': 'logistic', 'classifier__alpha': 0.001, 'classifier__hidden_layer_sizes': (100, 50, 25)}
ROC AUC Score: 0.7863
Best parameters for Extra Trees: {'classifier__bootstrap': True, 'classifier__max_depth': None, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 200}
ROC AUC Score: 0.7767


## Grid serach for scoring roc_auc & f1

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, roc_auc_score, f1_score
import joblib

# Define the frequency encoding function
def frequency_encoding(df, column, threshold=10):
    freq_counts = df[column].value_counts()
    freq_encoding = df[column].map(freq_counts)
    df[f'{column}_freq'] = freq_encoding
    
    # Group less frequent categories
    df[column] = df[column].apply(lambda x: 'Other' if freq_counts[x] < threshold else x)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    df = df.astype({
        "id": str,
        "name": str,
        "sex": str,
        "dob": str,
        "race": str,
        "juv_fel_count": int,
        "juv_misd_count": int,
        "juv_other_count": int,
        "priors_count": int,
        "c_case_number": str,
        "c_charge_degree": str,
        "c_charge_desc": str,
        "c_offense_date": str,
        "c_arrest_date": str,
        "c_jail_in": str
    })
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(0)  # Fill NaN with 0
    df['time_in_jail'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days  # Duration in jail till today
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek
    df['arrest_month'] = df['c_arrest_date'].dt.month
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek

    # Interaction features
    df['total_crimes_time_in_jail_interaction'] = df['total_adult_crimes'] * df['time_in_jail']
    df['total_crimes_time_in_jail_ratio'] = df['total_adult_crimes'] / (df['time_in_jail'] + 1)  # +1 to avoid division by zero
    df['total_crimes_age_at_unified_interaction'] = df['total_adult_crimes'] * df['age_at_unified_date']
    df['time_in_jail_age_interaction'] = df['time_in_jail'] * df['age_birth']
    df['total_crimes_offense_month_interaction'] = df['total_adult_crimes'] * df['offense_month']
    df['male_total_crimes_interaction'] = df['total_adult_crimes'] * (df['sex'] == 'Male').astype(int)
    df['charge_freq_total_crimes_interaction'] = df['c_charge_desc_freq'] * df['total_adult_crimes']
    df['day_in_jail_interaction'] = df['time_in_jail'] * df['arrest_day_of_week']
    
    return df


# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", "race", 
    "juv_fel_count", 
    "juv_misd_count", 
    "juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
    #"c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "age_birth",
    "time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "total_crimes_time_in_jail_interaction",
    "total_crimes_time_in_jail_ratio",
    "total_crimes_age_at_unified_interaction",
    "time_in_jail_age_interaction",
    "total_crimes_offense_month_interaction",
    "male_total_crimes_interaction",
    "charge_freq_total_crimes_interaction",
    "day_in_jail_interaction"
]

numerical_features = [
    "juv_fel_count", 
    "juv_misd_count", 
    "juv_other_count", 
    "total_adult_crimes",
    "time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", "age_birth",
    "c_charge_desc_freq",
    "total_crimes_time_in_jail_interaction",
    "total_crimes_time_in_jail_ratio",
    "total_crimes_age_at_unified_interaction",
    "time_in_jail_age_interaction",
    "total_crimes_offense_month_interaction",
    "male_total_crimes_interaction",
    "charge_freq_total_crimes_interaction",
    "day_in_jail_interaction"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
    #"c_charge_desc"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Define a dictionary of models with their respective parameter grids for hyperparameter tuning
models = {
    'Logistic Regression': {
        'model': LogisticRegression(solver='liblinear'),  # Change solver here
        'params': {
            'classifier__C': [0.1, 1.0, 10.0],
            'classifier__penalty': ['l1', 'l2']
        }
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4]
        }
    },
    'Random Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4],
            'classifier__bootstrap': [True, False]
        }
    },
    'Support Vector Machine': {
        'model': SVC(probability=True),
        'params': {
            'classifier__C': [0.1, 1.0, 10.0],
            'classifier__kernel': ['linear', 'rbf']
        }
    },
    'AdaBoost': {
        'model': AdaBoostClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 1.0]
        }
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__learning_rate': [0.01, 0.1, 1.0],
            'classifier__max_depth': [3, 5, 7]
        }
    },
    'K-Nearest Neighbors': {
        'model': KNeighborsClassifier(),
        'params': {
            'classifier__n_neighbors': [3, 5, 7],
            'classifier__weights': ['uniform', 'distance']
        }
    },
    'Multi-layer Perceptron': {
        'model': MLPClassifier(),
        'params': {
            'classifier__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50, 25)],
            'classifier__activation': ['logistic', 'relu'],
            'classifier__alpha': [0.0001, 0.001, 0.01]
        }
    },
    'Extra Trees': {
        'model': ExtraTreesClassifier(),
        'params': {
            'classifier__n_estimators': [50, 100, 200],
            'classifier__max_depth': [None, 5, 10, 20],
            'classifier__min_samples_split': [2, 5, 10],
            'classifier__min_samples_leaf': [1, 2, 4],
            'classifier__bootstrap': [True, False]
        }
    }
}

scoring = {'roc_auc': 'roc_auc', 'f1': 'f1'}

best_models = {}

for name, model_info in models.items():
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model_info['model'])])
    grid_search = GridSearchCV(pipeline, model_info['params'], cv=5, scoring=scoring, refit='roc_auc', error_score='raise')
    grid_search.fit(df_train[all_features], df_train[target])
    
    best_models[name] = {
        'best_params': grid_search.best_params_,
        'best_score_roc_auc': grid_search.cv_results_['mean_test_roc_auc'][grid_search.best_index_],
        'best_score_f1': grid_search.cv_results_['mean_test_f1'][grid_search.best_index_]
    }
    
    print(f'Best parameters for {name}: {grid_search.best_params_}')
    print(f'Best ROC AUC Score: {grid_search.cv_results_["mean_test_roc_auc"][grid_search.best_index_]:.4f}')
    print(f'Best F1 Score: {grid_search.cv_results_["mean_test_f1"][grid_search.best_index_]:.4f}')
    
# Print the best models summary
for model_name, model_details in best_models.items():
    print(f'{model_name} - Best Params: {model_details["best_params"]}')
    print(f'Best ROC AUC Score: {model_details["best_score_roc_auc"]:.4f}')
    print(f'Best F1 Score: {model_details["best_score_f1"]:.4f}')
    print('---')


## Grid serach for scoring roc_auc

In [140]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import joblib

# Define the frequency encoding function
def frequency_encoding(df, column, threshold=10):
    freq_counts = df[column].value_counts()
    freq_encoding = df[column].map(freq_counts)
    df[f'{column}_freq'] = freq_encoding
    
    # Group less frequent categories
    df[column] = df[column].apply(lambda x: 'Other' if freq_counts[x] < threshold else x)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    df = df.astype({
        "id": str,
        "name": str,
        "sex": str,
        "dob": str,
        "race": str,
        "juv_fel_count": int,
        "juv_misd_count": int,
        "juv_other_count": int,
        "priors_count": int,
        "c_case_number": str,
        "c_charge_degree": str,
        "c_charge_desc": str,
        "c_offense_date": str,
        "c_arrest_date": str,
        "c_jail_in": str
    })
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(0)  # Fill NaN with 0
    df['time_in_jail'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days  # Duration in jail till today
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek
    df['arrest_month'] = df['c_arrest_date'].dt.month
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek

    # Interaction features
    df['fel_misd_interaction'] = df['juv_fel_count'] * df['juv_misd_count']
    
    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", "race", 
    "juv_fel_count", 
    "juv_misd_count", 
    "juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
   # "c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "age_birth",
    #"time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    #"fel_misd_interaction"
]

numerical_features = [
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count", 
    "total_adult_crimes",
    #"time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date",
    "age_birth",
    "c_charge_desc_freq",
    #"fel_misd_interaction"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
    #"c_charge_desc"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Save the updated pipeline
#joblib.dump(pipeline, 'best_model_pipeline.pkl')

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Cross-validate the updated pipeline
cv_roc_auc_scores = cross_val_score(pipeline, df[all_features], df[target], cv=5, scoring='roc_auc')
print(f'Cross-Validated ROC-AUC Scores: {cv_roc_auc_scores}')
print(f'Average Cross-Validated ROC-AUC: {cv_roc_auc_scores.mean():.2f}')

# Extract feature importances
model = pipeline.named_steps['classifier']
importances = model.feature_importances_

# Get feature names from preprocessor
num_features = preprocessor.transformers_[0][2]
cat_features = preprocessor.transformers_[1][1].named_steps['onehot'].get_feature_names_out(categorical_features)
feature_names = np.concatenate([num_features, cat_features])

# Create a DataFrame for feature importances
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Sort the DataFrame by importance
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Display top 10 and bottom 10 features
print("Top 10 Features:\n", feature_importances.head(10))
print("\nLast 10 Features:\n", feature_importances.tail(10))


Accuracy: 0.75
Precision: 0.79
Recall: 0.60
F1-Score: 0.68
ROC-AUC: 0.82
Confusion Matrix:
[[559  85]
 [210 317]]
Cross-Validated ROC-AUC Scores: [0.80735393 0.82963205 0.81723015 0.80199272 0.79261207]
Average Cross-Validated ROC-AUC: 0.81
Top 10 Features:
                 Feature  Importance
1          time_in_jail    0.557319
0    total_adult_crimes    0.206429
8             age_birth    0.074816
7   age_at_unified_date    0.073786
2      total_juv_crimes    0.050934
9    c_charge_desc_freq    0.007355
11             sex_Male    0.004911
3         offense_month    0.004444
6    arrest_day_of_week    0.004113
5          arrest_month    0.004095

Last 10 Features:
                   Feature  Importance
10             sex_Female    0.003776
4     offense_day_of_week    0.003734
12  race_African-American    0.003650
19      c_charge_degree_M    0.000344
14         race_Caucasian    0.000093
17             race_Other    0.000086
18      c_charge_degree_F    0.000064
16   race_Native Amer

## Model with SMOTE

In [139]:
#WITH SMOTE

import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Use imblearn's pipeline for SMOTE
import joblib

# Define the frequency encoding function
def frequency_encoding(df, column, threshold=10):
    freq_counts = df[column].value_counts()
    freq_encoding = df[column].map(freq_counts)
    df[f'{column}_freq'] = freq_encoding
    
    # Group less frequent categories
    df[column] = df[column].apply(lambda x: 'Other' if freq_counts[x] < threshold else x)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    df = df.astype({
        "id": str,
        "name": str,
        "sex": str,
        "dob": str,
        "race": str,
        "juv_fel_count": int,
        "juv_misd_count": int,
        "juv_other_count": int,
        "priors_count": int,
        "c_case_number": str,
        "c_charge_degree": str,
        "c_charge_desc": str,
        "c_offense_date": str,
        "c_arrest_date": str,
        "c_jail_in": str
    })
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(0)  # Fill NaN with 0
    df['time_in_jail'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days  # Duration in jail till today
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek
    df['arrest_month'] = df['c_arrest_date'].dt.month
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek

    # Interaction features
    df['fel_misd_interaction'] = df['juv_fel_count'] * df['juv_misd_count']
    
    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", "race", 
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
    #"c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "age_birth",
    #"time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    #"fel_misd_interaction"
]

numerical_features = [
    #"juv_fel_count", 
    #"juv_misd_count", 
    #"juv_other_count", 
    "total_adult_crimes",
    #"time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", "age_birth",
    "c_charge_desc_freq",
    #"fel_misd_interaction"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
    #"c_charge_desc"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

# Create the pipeline with SMOTE
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Cross-validate the updated pipeline
cv_roc_auc_scores = cross_val_score(pipeline, df[all_features], df[target], cv=5, scoring='roc_auc')
print(f'Cross-Validated ROC-AUC Scores: {cv_roc_auc_scores}')
print(f'Average Cross-Validated ROC-AUC: {cv_roc_auc_scores.mean():.2f}')


Accuracy: 0.74
Precision: 0.75
Recall: 0.65
F1-Score: 0.69
ROC-AUC: 0.82
Confusion Matrix:
[[528 116]
 [185 342]]
Cross-Validated ROC-AUC Scores: [0.8077062  0.82913888 0.81670188 0.80360099 0.79463643]
Average Cross-Validated ROC-AUC: 0.81


## NEW FEATURES

In [151]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import joblib

# Define the frequency encoding function
def frequency_encoding(df, column, threshold=10):
    freq_counts = df[column].value_counts()
    freq_encoding = df[column].map(freq_counts)
    df[f'{column}_freq'] = freq_encoding
    
    # Group less frequent categories
    df[column] = df[column].apply(lambda x: 'Other' if freq_counts[x] < threshold else x)
    return df

# Define the preprocess_data function
def preprocess_data(df):
    df = df.astype({
        "id": str,
        "name": str,
        "sex": str,
        "dob": str,
        "race": str,
        "juv_fel_count": int,
        "juv_misd_count": int,
        "juv_other_count": int,
        "priors_count": int,
        "c_case_number": str,
        "c_charge_degree": str,
        "c_charge_desc": str,
        "c_offense_date": str,
        "c_arrest_date": str,
        "c_jail_in": str
    })
    df['dob'] = pd.to_datetime(df['dob'])
    df['c_offense_date'] = pd.to_datetime(df['c_offense_date'])
    df['c_arrest_date'] = pd.to_datetime(df['c_arrest_date'])
    df['c_jail_in'] = pd.to_datetime(df['c_jail_in'])

    # Create unified_date
    df['unified_date'] = df['c_arrest_date'].combine_first(df['c_offense_date'])

    # Feature engineering
    df['age_birth'] = (pd.to_datetime('today') - df['dob']).dt.days // 365
    df['age_at_unified_date'] = (df['unified_date'] - df['dob']).dt.days // 365
    df['time_offense_arrest'] = (df['c_arrest_date'] - df['c_offense_date']).dt.days.fillna(0)  # Fill NaN with 0
    df['time_in_jail'] = (pd.to_datetime('today') - df['c_jail_in']).dt.days  # Duration in jail till today
    df['total_juv_crimes'] = df['juv_fel_count'] + df['juv_misd_count'] + df['juv_other_count']
    df['total_adult_crimes'] = df['priors_count'] - df['total_juv_crimes']

    # Apply frequency encoding and group less frequent categories
    df = frequency_encoding(df, 'c_charge_desc')

    # Create age bins
    bins = [18, 25, 35, 45, 55, 65, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '56-65', '66+']
    df['age_group'] = pd.cut(df['age_birth'], bins=bins, labels=labels, right=False)
    
    # Extract more granular date features
    df['offense_month'] = df['c_offense_date'].dt.month
    df['offense_day_of_week'] = df['c_offense_date'].dt.dayofweek
    df['arrest_month'] = df['c_arrest_date'].dt.month
    df['arrest_day_of_week'] = df['c_arrest_date'].dt.dayofweek

    # Interaction features
    df['total_crimes_time_in_jail_interaction'] = df['total_adult_crimes'] * df['time_in_jail']
    df['total_crimes_time_in_jail_ratio'] = df['total_adult_crimes'] / (df['time_in_jail'] + 1)  # +1 to avoid division by zero
    df['total_crimes_age_at_unified_interaction'] = df['total_adult_crimes'] * df['age_at_unified_date'] 
    df['time_in_jail_age_interaction'] = df['time_in_jail'] * df['age_birth']
    df['total_crimes_offense_month_interaction'] = df['total_adult_crimes'] * df['offense_month']
    df['male_total_crimes_interaction'] = df['total_adult_crimes'] * (df['sex'] == 'Male').astype(int)
    df['charge_freq_total_crimes_interaction'] = df['c_charge_desc_freq'] * df['total_adult_crimes']
    df['day_in_jail_interaction'] = df['time_in_jail'] * df['arrest_day_of_week']
    
    return df

# Load data (ensure df is loaded before this step)
# df = pd.read_csv('your_data.csv')  # Example loading method

# Apply preprocessing to the entire dataset
df = preprocess_data(df)

# Split data into train and test sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Define features
all_features = [
    "sex", "race", 
    "juv_fel_count", 
    "juv_misd_count", 
    "juv_other_count",
    "total_adult_crimes", 
    "c_charge_degree", 
    "c_charge_desc", 
    "c_charge_desc_freq",
    "age_at_unified_date", 
    "age_birth",
    "time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
   # "total_crimes_time_in_jail_interaction",
   # "total_crimes_time_in_jail_ratio",
    #"total_crimes_age_at_unified_interaction",
    #"time_in_jail_age_interaction",
    #"total_crimes_offense_month_interaction",
    #"male_total_crimes_interaction",
    #"charge_freq_total_crimes_interaction",
    "day_in_jail_interaction"
]

numerical_features = [
    "juv_fel_count", 
    "juv_misd_count", 
    "juv_other_count", 
    "total_adult_crimes",
    "time_offense_arrest", 
    "time_in_jail", 
    "total_juv_crimes",
    "offense_month", 
    "offense_day_of_week", 
    "arrest_month", 
    "arrest_day_of_week",
    "age_at_unified_date", "age_birth",
    "c_charge_desc_freq",
    #"total_crimes_time_in_jail_interaction",
    #"total_crimes_time_in_jail_ratio",
    #"total_crimes_age_at_unified_interaction",
    #"time_in_jail_age_interaction",
    #"total_crimes_offense_month_interaction",
    #"male_total_crimes_interaction",
    #"charge_freq_total_crimes_interaction",
    "day_in_jail_interaction"
]

categorical_features = [
    "sex", 
    "race", 
    "c_charge_degree", 
    "c_charge_desc"
]

target = 'is_recid'

# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Use the best model parameters found earlier
best_model_params = {
    'learning_rate': 0.01,
    'max_depth': 3,
    'n_estimators': 300,
    'subsample': 0.8,
    'random_state': 42
}

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(**best_model_params))
])

# Train the updated pipeline
X_train = df_train[all_features]
y_train = df_train[target]
pipeline.fit(X_train, y_train)

# Save the updated pipeline
joblib.dump(pipeline, 'best_model_pipeline.pkl')

# Evaluate the updated pipeline on the test set
X_test = df_test[all_features]
y_test = df_test[target]
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')
print(f'ROC-AUC: {roc_auc:.2f}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Cross-validate the updated pipeline
cv_roc_auc_scores = cross_val_score(pipeline, df[all_features], df[target], cv=5, scoring='roc_auc')
print(f'Cross-Validated ROC-AUC Scores: {cv_roc_auc_scores}')
print(f'Average Cross-Validated ROC-AUC: {cv_roc_auc_scores.mean():.2f}')


Accuracy: 0.75
Precision: 0.79
Recall: 0.61
F1-Score: 0.68
ROC-AUC: 0.82
Confusion Matrix:
[[558  86]
 [208 319]]
Cross-Validated ROC-AUC Scores: [0.80758584 0.82852974 0.81712303 0.80298762 0.79320159]
Average Cross-Validated ROC-AUC: 0.81
